In [38]:
import collections
import json
import os
import re

import ahocorasick
import openai
import pdfplumber
from pdfminer.high_level import extract_text

dataDir = "../data/"
dataName = "Deep Learning.pdf"

# 设置 api


In [5]:
openai.api_base = "https://api.chatanywhere.com.cn/v1"
# openai.api_base = 'https://api.chatanywhere.cn/v1'
openai.api_key = "sk-D1u13WweY1LhWLqv95Ml7e3y8f8ToSfsTkGnlgvSQLqZJptC"

# 一，读取数据


In [6]:
# 不选择用这种方式读取，用pdfplumber更好
# text = extract_text('./Deep Learning (Ian Goodfellow, Yoshua Bengio, Aaron Courville).pdf')

## 生成目录架构

- 生成**章节名称**与**章节序号**的对应：`name_dict`
- 以及**章节序号**与**章节页码**范围的对应：`index_dict`


In [7]:
with pdfplumber.open(dataDir + dataName) as f:
    # 目录架构生成
    c, p, n = [], [], []
    for i in range(7):
        page = f.pages[i]
        text = page.extract_text()
        text_split = text.split("\n")
        for i in text_split:
            if bool(re.match("[0-9]+\.[0-9]+", i.split(" ")[0])):
                c.append(i.split(" ")[0])
                p.append(i.split(" ")[-1])
            if bool(re.match("[0-9]+", i.split(" ")[0])):
                for j in i.split(" "):
                    if bool(re.match("[A-Za-z]+", j)):
                        n.append((i.split(" ")[0], j))

### name_dict


In [8]:
name_dict = {}
for i, j in n:
    if i in name_dict:
        name_dict[i] = name_dict.get(i, "") + " " + j
    else:
        name_dict[i] = name_dict.get(i, "") + j

### index_dict


In [9]:
p_range = list(zip(p, p[1:]))
p_range.append((720, 800))
c_p_range = list(zip(c, p_range))
index_dict = collections.defaultdict(list)
for k, v in c_p_range:
    index_dict[k.split(".")[0]].append((k, v))

## 生成内容表

- 段落内容表：`content_dict`
- 句子内容表：`sentence_dict`


In [10]:
with pdfplumber.open(dataDir + dataName) as f:
    content_dict = collections.defaultdict(list)

    sentence_dict = collections.defaultdict(list)

    for k, v in index_dict.items():
        for i in v:
            page_range = i[-1]

            for j in range(int(page_range[0]), int(page_range[1])):
                page = f.pages[j]

                text = page.extract_text().replace("\n", " ")

                content_dict[i[0]].append(text)

                text_split = text.split(".")

                for t in text_split:
                    sentence_dict[i[0]].append(t)

        break

# 二，Build Relations

## 目录 + 前置

In [30]:
def catalogue_relations(dataName, name_dict=None, relation_type=["目录", "前置"]):
    c_relations = []
    p_relations = []
    if not name_dict:
        name_dict = {}
    for k, v in name_dict.items():
        if bool(re.match("[0-9]+\.[0-9]+", k)):
            p_relations.append(
                {
                    "head": k.split(".")[0] + " " + name_dict[k.split(".")[0]],
                    "type": relation_type[1],
                    "tail": k + " " + v,
                }
            )
        else:
            c_relations.append(
                {
                    "head": dataName,
                    "type": relation_type[0],
                    "tail": k + " " + v,
                }
            )
    return p_relations, c_relations

In [31]:
p_relations, c_relations = catalogue_relations(
    dataName=dataName.split(".")[0], name_dict=name_dict
)

## PlanA
根据Section抽取关系，可以比较容易且准确的判断出包含、段落共现、句子共现的关系

In [55]:
with open(os.path.join(dataDir, "chapter1_read_by_sections.json"), "r") as f:
    chapter1_read_by_section = json.load(f)

In [55]:
entity_dict = {}
for i in chapter1_read_by_section:
    for k, v in i.items():
        entity_set = set()
        for relation in v:
            if relation['head'] not in entity_set:
                entity_set.add(relation['head'])
            if relation['tail'] not in entity_set:
                entity_set.add(relation['tail'])
        entity_dict[k] = entity_set

### AC自动机

In [117]:
def build(patterns):
    trie = ahocorasick.Automaton()
    for index, word in enumerate(patterns):
        trie.add_word(word, (index, word))
    trie.make_automaton()
    return trie

### 包含

In [57]:
include_relations = []
for k, v in entity_dict.items():
    for entity in v:
        include_relations.append(
            {
                "head": k,
                "type": "包含",
                "tail": entity,
            }
        )

In [59]:
include_relations[0]

{'head': '1.1 Who Should Read This Book?',
 'type': '包含',
 'tail': 'machine learning'}

### 段落共现

In [121]:
def co_presence(entity_dict, relation_type):
    if relation_type == "sentence":
        text_dict = sentence_dict
    elif relation_type == "paragraph":
        text_dict = content_dict
    relations = []
    for k, v in entity_dict.items():
        patterns = list(v)
        for text in text_dict[k.split(" ")[0]]:
            trie = build(patterns)
            word_set = set(word[1][1] for word in trie.iter(text))
            if len(word_set) > 1:
                relations.append(word_set)
    return relations

In [122]:
paragraph_relations = co_presence(entity_dict, 'paragraph')

In [123]:
paragraph_relations[8]

{'Applied Math',
 'Applied Math and Machine Learning Basics',
 'CNN',
 'Deep Networks',
 'Machine Learning',
 'RNNs'}

### 句子共现

In [124]:
sentence_relations = co_presence(entity_dict, 'sentence')

In [125]:
sentence_relations[1]

{'algorithm', 'machine learning'}

### 频繁项集

In [118]:
# ???

## PlanB
按章抽取关系/全书抽取关系  
包含：查看实体是否存在与一个section  
段落共现、句子共现：设定阈值，根据实体与实体之间出现的距离判断  
缺点是不准确  

In [119]:
entity_set = set()
for i in chapter1_read_by_section:
    for k, v in i.items():
        for relation in v:
            if relation["head"] not in entity_set:
                entity_set.add(relation["head"])
            if relation["tail"] not in entity_set:
                entity_set.add(relation["tail"])

In [146]:
def co_presence(entity_dict, relation_type, threshold_value=[50, 500]):
    patterns = list(entity_dict)
    trie = build(patterns)
    relations = []
    for section in ["1.1", "1.2"]:
        lines = ""
        for line in content_dict[section]:
            lines += line
        word_set = set(word for word in trie.iter(text))
        if relation_type == "include":
            for word in word_set:
                relations.append(
                    {
                        "head": section,
                        "type": "包含",
                        "tail": word[1][1],
                    }
                )
        else:
            if relation_type == "sentence":
                threshold = range(0, threshold_value[0])
                r_type = "句子共现"
            elif relation_type == "paragraph":
                threshold = range(threshold_value[0], threshold_value[1])
                r_type = "段落共现"
            for word_head in word_set:
                for word_tail in word_set:
                    if (
                        word_head[0] - word_tail[0] in threshold
                        and word_head[1][1] != word_tail[1][1]
                    ):
                        relations.append(
                            {
                                "head": word_head[1][1],
                                "type": r_type,
                                "tail": word_tail[1][1],
                            }
                        )
    return relations

### 包含

In [148]:
include_relations = co_presence(entity_set, 'include')

### 句子共现

In [150]:
sentence_relations = co_presence(entity_set, 'sentence')

### 段落共现

In [152]:
paragraph_relations = co_presence(entity_set, 'paragraph')

## 结果展示

In [33]:
p_relations[0]

{'head': '1 Introduction',
 'type': '前置',
 'tail': '1.1 Who Should Read This Book?'}

In [27]:
c_relations[0]

{'head': 'Deep Learning', 'type': '目录', 'tail': '1 Introduction'}

In [149]:
include_relations[0]

{'head': '1.1', 'type': '包含', 'tail': 'deep learning'}

In [158]:
sentence_relations[1]

{'head': 'rectified linear unit', 'type': '句子共现', 'tail': 'neuron'}

In [159]:
paragraph_relations[1]

{'head': 'deep learning', 'type': '段落共现', 'tail': 'machine learning'}

# 测试

In [ ]:
# relations = ["目录", "前置", "句子共现", "段落共现", "频繁项集"]  # pending

# conversation_list = [
#     {
#         "role": "system",
#         "content": "请只提取文本中的命名实体，格式为[{entity_A}, {entity_B}, ...], 不要返回任何其他内容",
#     }
# ]

# bot = Chat(conversation_list)
# answer = bot.ask(content_dict["1.1"][1])

In [ ]:
# res_tiny = [i.strip() for i in re.sub('"|}|{||\[|\]', "", answer).split(",")]

# 一些其他尝试

N - Gram


In [ ]:
# 暂时没有探索结果
# all_text = re.sub('[^A-Za-z0-9\.]+', ' ', text).lower().split(' ')
# ng1 = collections.defaultdict(int)
# ng2 = collections.defaultdict(int)
# ng3 = collections.defaultdict(int)
# ng4 = collections.defaultdict(int)
# for i, j in enumerate(all_text):
#     ng1[j] += 1
#     if i > 0: ng2[(all_text[i-1], j)] += 1
#     if i > 1: ng3[(all_text[i-2], all_text[i-1], j)] += 1
#     if i > 2: ng4[(all_text[i-3], all_text[i-2], all_text[i-1], j)] += 1

# def sort_feq(dic):
#     return sorted([(k, v) for k, v in dic.items()], key=lambda x: x[1], reverse=True)

### 定义 Chat 类

In [ ]:
# def total_counts(response):
#     tokens_nums = int(response["usage"]["total_tokens"])
#     price = 0.002 / 1000
#     cost = "{:.5f}".format(price * tokens_nums * 7.5)
#     print(f"tokens: {tokens_nums}, cost: {cost}")

#     return float(cost)

In [ ]:
# class Chat:
#     def __init__(self, conversation_list=[]):
#         self.conversation_list = conversation_list
#         self.costs_list = []

#     def show_conversation(self, msg_list):
#         for msg in msg_list[-2:]:
#             if msg["role"] == "user":
#                 pass
#             else:
#                 message = msg["content"]
#                 print(f"\U0001f47D: {message}\n")
#             print()

#     def ask(self, prompt):
#         self.conversation_list.append({"role": "user", "content": prompt})
#         openai.api_key = "sk-D1u13WweY1LhWLqv95Ml7e3y8f8ToSfsTkGnlgvSQLqZJptC"
#         response = openai.ChatCompletion.create(
#             model="gpt-3.5-turbo", messages=self.conversation_list
#         )
#         answer = response.choices[0].message["content"]

#         self.conversation_list.append({"role": "assistant", "content": answer})
#         self.show_conversation(self.conversation_list)

#         cost = total_counts(response)
#         self.costs_list.append(cost)
#         return answer
#         print()